In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Analyzing ACRONYM data

In [46]:
import pandas as pd
import numpy as np

# path = '/data/manifolds/acronym/bullet_grasps/fullrun_grasps.csv'
path = '/data/manifolds/acronym/bullet_grasps/grasps.csv'

df = pd.read_csv(path)
# print(df)
both_true = df.query('flex_success==1 & pybullet_success==1')
print("both true", len(both_true))
flex_true = df.query('flex_success==1')
print("flex true", len(flex_true))
bullet_true = df.query('pybullet_success==1')
print("bullet true", len(bullet_true))

print(f"true bullet positives over flex positives: {len(both_true) / len(flex_true):.3f}")
print(f"true bullet positives over all positives: {len(both_true) / len(bullet_true):.3f}")

both_false = df.query('flex_success==0 & pybullet_success==0')
print("both false", len(both_false))
flex_false = df.query('flex_success==0')
print("flex false", len(flex_false))
bullet_false = df.query('pybullet_success==0')
print("bullet false", len(bullet_false))

print(f"true bullet negatives over flex negatives: {len(both_false) / len(flex_false):.3f}")
print(f"true bullet negatives over all negatives: {len(both_false) / len(bullet_false):.3f}")



both true 240
flex true 1139
bullet true 247
true bullet positives over flex positives: 0.211
true bullet positives over all positives: 0.972
both false 854
flex false 861
bullet false 1753
true bullet negatives over flex negatives: 0.992
true bullet negatives over all negatives: 0.487


In [47]:
print("Error reason")
print(df.loc[:, 'err_info'].value_counts())

Error reason
grasp     1740
linear      13
Name: err_info, dtype: int64


# Analyzing planning trials

In [12]:
exp_suffix = 'infgrasps_OMG'

out_path = '/home/exx/projects/manifolds/OMG-Planner/output_videos/'

exp_paths = []
for date_path in os.listdir(out_path):
    if 'dbg' in date_path or 'old_runs' in date_path:
        continue
    for exp_path in os.listdir(f'{out_path}/{date_path}'):
        if exp_suffix in exp_path:
            exp_paths.append(f'{out_path}/{date_path}/{exp_path}')

if len(exp_paths) != 10:
    print(f"Warning: {len(exp_paths)} total runs for {exp_suffix}: {len(exp_paths)}")
else:
    print("10 runs")

runs_metrics = {
    'exec': [],
    'plan': [],
    'smth': [],
    'coll': [],
    'time': []
}
for exp_path in exp_paths:
    scene_paths = sorted(os.listdir(exp_path), key=lambda x: int(x.split('_')[1]))
    if len(scene_paths) < 100:
        print(f"Skipping {exp_path}")
        continue

    exp_metrics = {
        'exec': [],
        'plan': [],
        'smth': [],
        'coll': [],
        'time': [],
    }
    for i, scene_path in enumerate(scene_paths):
        try:
            data = np.load(f'{exp_path}/{scene_path}/data.npy', allow_pickle=True, encoding='latin1')
        except Exception as e:
            print(e)
            print(f"Stopping early at {i}")
            break
        rew, info, _ = data
        exp_metrics['exec'].append(rew)
        exp_metrics['plan'].append(1 if info[-1]['terminate'] else 0)
        exp_metrics['smth'].append(info[-1]['smooth'])
        exp_metrics['coll'].append(info[-1]['obs'])
        exp_metrics['time'].append(info[-1]['time'])
#     print(exp_metrics['time'])
    runs_metrics['exec'].append(np.mean(exp_metrics['exec']))
    runs_metrics['plan'].append(np.mean(exp_metrics['plan']))
    runs_metrics['smth'].append(np.mean(exp_metrics['smth']))
    runs_metrics['coll'].append(np.mean(exp_metrics['coll']))
    runs_metrics['time'].append(np.mean(exp_metrics['time']))
mean_exec = np.mean(runs_metrics['exec'])
mean_plan = np.mean(runs_metrics['plan'])
mean_smth = np.mean(runs_metrics['smth'])
mean_coll = np.mean(runs_metrics['coll'])
mean_time = np.mean(runs_metrics['time'])
print(f'{mean_exec:.2f}+/-{np.std(runs_metrics["exec"]):.2f}')
print(f'{mean_plan:.2f}+/-{np.std(runs_metrics["plan"]):.2f}')
print(f'{mean_smth:.2f}+/-{np.std(runs_metrics["smth"]):.2f}')
print(f'{mean_coll:.2f}+/-{np.std(runs_metrics["coll"]):.2f}')
print(f'{mean_time:.2f}+/-{np.std(runs_metrics["time"]):.2f}')

print("To do: formatting; save/print the runs used")

10 runs
0.15+/-0.04
0.95+/-0.02
14.33+/-0.64
20.87+/-4.52
2.37+/-0.05
To do: formatting; save/print the runs used
